# Ingestão Dinâmica - Dados SIAPE

---

## Objetivo
Este notebook é responsável pela extração automatizada dos dados de servidores públicos federais diretamente da API do **Portal da Transparência**. Ele baixa os arquivos brutos, filtra os domínios de interesse e os organiza em nossa camada de Staging (Databricks Volumes) preparando o terreno para a arquitetura Medallion.

## Regras de Negócio Implementadas
* **Histórico Dinâmico Autônomo:** O pipeline não depende de datas hardcoded. Ele possui um motor temporal que identifica a data de execução, calcula a **posição atual (M-2)** (devido ao delay de fechamento da folha do governo) e itera automaticamente para capturar **3 meses de histórico adicionais** (M-3, M-4 e M-5).

## Engenharia e Performance
1. **Prevenção de OOM (Out of Memory):** Como os pacotes ZIP do governo chegam a ter múltiplos Gigabytes, a requisição HTTP é feita utilizando `stream=True`, baixando os dados em blocos.
2. **Processamento In-Memory (`BytesIO`):** Para evitar gargalos de I/O em disco, o arquivo `.zip` é aberto e descompactado diretamente na **Memória RAM** de forma transiente.
3. **Filtro de Domínio:** O script ignora arquivos irrelevantes do governo (ex: pensionistas, militares) e extrai apenas as tabelas exigidas pelo case: `Cadastro`, `Remuneração`, `Afastamentos` e `Observações`.
4. **Hive Partitioning Físico:** Os arquivos extraídos são salvos no Data Lake simulando uma estrutura nativa de partições (`/dominio/posicao_base=YYYYMM/arquivo.csv`), o que habilitará o *Native Partition Discovery* na Camada Bronze.

---
**Origem:** API REST (Portal da Transparência)

**Destino:** Databricks Unity Catalog Volumes (`/Volumes/workspace/default/staging-siape-tables`)

In [0]:
import os
import requests
import zipfile
import logging
from io import BytesIO
from datetime import datetime

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

VOLUME_BASE_PATH = "/Volumes/workspace/default/staging-siape-tables"

BASE_URL = "https://portaldatransparencia.gov.br/download-de-dados/servidores"

DOMAINS_OF_INTEREST = ["cadastro", "remuneracao", "afastamentos", "observacoes"]

def get_dynamic_target_months(reference_date=None):
    """
    Calcula os meses alvo da ingestão dinamicamente sem hardcode.
    Regra: M-2 (Posição Atual do SIAPE) e 3 meses para trás (M-3, M-4, M-5).
    """
    if not reference_date:
        reference_date = datetime.now()
        
    target_months = []
    
    # Converte o ano/mês para um valor linear para facilitar subtração entre anos
    base_val = reference_date.year * 12 + reference_date.month - 1
    
    # Offsets: 2 (M-2), 3 (M-3), 4 (M-4), 5 (M-5) -> 4 meses totais de safra
    for offset in [2, 3, 4, 5]:
        target_val = base_val - offset
        y = target_val // 12
        m = (target_val % 12) + 1
        target_months.append(f"{y}{m:02d}")
        
    return target_months

def download_and_extract_to_volumes(ano_mes: str):
    target_url = f"{BASE_URL}/{ano_mes}_Servidores_SIAPE"
    logger.info(f"Iniciando download da API para safra {ano_mes}: {target_url}")
    
    try:
        response = requests.get(target_url, stream=True, timeout=120)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        logger.error(f"❌ Erro ao baixar os dados do mês {ano_mes}. A safra pode não estar disponível: {e}")
        return

    logger.info(f"Download de {ano_mes} concluído! Extraindo arquivos para os Volumes...")
    
    try:
        with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
            for file_name in zip_ref.namelist():
                
                file_name_lower = file_name.lower()
                if any(domain in file_name_lower for domain in DOMAINS_OF_INTEREST):
                    
                    domain_folder = file_name.split('_')[1].split('.')[0].lower()
                    if domain_folder == 'remuneracao': domain_folder = 'remuneracao'
                    if domain_folder == 'observacoes': domain_folder = 'observacoes'
                        
                    partition_path = os.path.join(VOLUME_BASE_PATH, domain_folder, f"posicao_base={ano_mes}")
                    os.makedirs(partition_path, exist_ok=True)
                    
                    final_file_path = os.path.join(partition_path, file_name)
                    logger.info(f" -> Escrevendo: {final_file_path}")
                    
                    with zip_ref.open(file_name) as source_file:
                        with open(final_file_path, "wb") as target_file:
                            target_file.write(source_file.read())
                            
        logger.info(f"✅ Ingestão do mês {ano_mes} concluída com sucesso nos Volumes!")
        
    except zipfile.BadZipFile:
        logger.error("❌ O arquivo baixado não é um ZIP válido ou o portal bloqueou a requisição.")
    except Exception as e:
        logger.error(f"❌ Erro inesperado durante a extração: {e}")


if __name__ == "__main__":
    logger.info("Iniciando Pipeline de Ingestão Automática (Extract & Load)...")
    
    meses_para_ingerir = get_dynamic_target_months()
    logger.info(f"Safras identificadas para processamento (M-2 a M-5): {meses_para_ingerir}")

    for mes in meses_para_ingerir:
        download_and_extract_to_volumes(mes)
        
    logger.info("Pipeline de Ingestão Finalizado!")

2026-02-25 22:52:16,645 - Iniciando Pipeline de Ingestão Automática (Extract & Load)...
2026-02-25 22:52:16,646 - Safras identificadas para processamento (M-2 a M-5): ['202512', '202511', '202510', '202509']
2026-02-25 22:52:16,646 - Iniciando download da API para safra 202512: https://portaldatransparencia.gov.br/download-de-dados/servidores/202512_Servidores_SIAPE
2026-02-25 22:52:17,333 - Download de 202512 concluído! Extraindo arquivos para os Volumes...
2026-02-25 22:52:26,786 -  -> Escrevendo: /Volumes/workspace/default/staging-siape-tables/afastamentos/posicao_base=202512/202512_Afastamentos.csv
2026-02-25 22:52:27,871 -  -> Escrevendo: /Volumes/workspace/default/staging-siape-tables/cadastro/posicao_base=202512/202512_Cadastro.csv
2026-02-25 22:52:30,196 -  -> Escrevendo: /Volumes/workspace/default/staging-siape-tables/observacoes/posicao_base=202512/202512_Observacoes.csv
2026-02-25 22:52:30,401 -  -> Escrevendo: /Volumes/workspace/default/staging-siape-tables/remuneracao/posi